In [1]:
pwd

'/opt/ml/final-project-level3-cv-02/airflow/dags'

In [2]:
cd /opt/ml/final-project-level3-cv-02/airflow

/opt/ml/final-project-level3-cv-02/airflow


In [14]:
import pendulum
from datetime import time, timedelta, datetime

from airflow import DAG 
from airflow.utils.dates import days_ago
from airflow.operators.bash import BashOperator
from airflow.providers.mysql.operators.mysql import MySqlOperator
from airflow.operators.python import PythonOperator
from airflow.models.variable import Variable

import os
import pymysql
import pandas as pd
from sqlalchemy import create_engine
from collections import defaultdict

# .env
from dotenv import load_dotenv
load_dotenv(dotenv_path = "../../.env")
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_PORT = os.getenv('MYSQL_PORT')
MYSQL_DBNAME = os.getenv('MYSQL_DB')
MYSQL_SERVER = os.getenv('MYSQL_SERVER')

In [15]:
print(MYSQL_USER)

cv02


In [8]:
!ls

__pycache__    hello_world.py  scripts	   user_statics.py
connect_db.py  inference.py    test.ipynb


In [9]:
!ls ../..

age_gender_classifier  airflow.db  back-end  front-end	sql_app
airflow		       babygan	   docker    logs


In [2]:
AIRFLOW_HOME = ' /opt/ml/final-project-level3-cv-02/airflow'
INFER_RESULT_PATH = os.path.join(AIRFLOW_HOME, 'infer_result')

def connect_db_get_data():
    MYSQL_USER_ID = os.getenv('MYSQL_USER_ID')
    MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
    MYSQL_PORT = os.getenv('MYSQL_PORT')
    MYSQL_DBNAME = os.getenv('MYSQL_DBNAME')
    MYSQL_SERVER = os.getenv('MYSQL_SERVER')

    print(MYSQL_USER_ID, MYSQL_PORT, type(MYSQL_PORT))

    DB = pymysql.connect(host=MYSQL_SERVER, user=MYSQL_USER_ID, passwd=MYSQL_PASSWORD, port=int(MYSQL_PORT), charset='utf8')

    data = pd.read_sql('''
    select * from cv02.inference_result''', con = DB)
    print(data.shape[0])

    os.makedirs(INFER_RESULT_PATH, exist_ok=True)
    data.to_csv(f'{INFER_RESULT_PATH}/infer_result_latest.csv', index=False)
    print(f'save inference_result_latest in {INFER_RESULT_PATH}')
    return data
   

def calculate_bounce_rate(data):
    print(f'calculate bounce rate after click inference button')
    df_log = pd.read_csv(f'{INFER_RESULT_PATH}/infer_result_latest.csv', parse_dates = ['created_time'])
    df_statics = pd.DataFrame(columns = ['ID', 'age', 'gender', 'inference_bounce_rate'])

    # calculate inference_bounce_rate
    # total_b_rate = len(df_log['complete' == False]) / len(df_log) # inference_bounce_rate -> #bounce / #click_inference_button = #complete==False / #ID
    df_statics['ID'] = [i for i in range(1,11)]

    # gender - age 
    gen_age = defaultdict(str)
    gender = ['male', 'female']
    age = [0, 20,30,40,50, 100]
    gen_age['female'] = []
    gen_age['male'] = []
    for g in gender:
        for a_i in range(1, len(age)-1):
            condition = (df_log.gender == g) & (age[a_i] <= df_log.age < age[a_i+1])
            print(len(df_log[condition]))
            gen_age[g].append(len(df_log[condition]))

    print(gen_age)

    return gen_age

def data_to_db(INFER_RESULT_PATH):
    DB = pymysql.connect(host=MYSQL_SERVER, user=MYSQL_USER_ID, passwd=MYSQL_PASSWORD, port=int(MYSQL_PORT), charset='utf8')

    infer_df = pd.read_csv(INFER_RESULT_PATH)

    engine = create_engine()
    engine_conn = engine.connect(f'mysql+pymysql://{MYSQL_USER_ID}:{MYSQL_PASSWORD}@{MYSQL_SERVER}:{MYSQL_PORT}/{MYSQL_DBNAME}?charset=utf8mb4')
    infer_df.to_sql('inference_bounce_rate', engine_conn, if_exists='replace', index=None)
    engine_conn.close()
    engine.dispose()

In [3]:
data = connect_db_get_data()

None 3306 <class 'str'>


OperationalError: (1045, "Access denied for user 'root'@'49.50.163.52' (using password: YES)")

In [93]:
data

,id,father_url,mother_url,baby_url,created_time,comment,closed_at,complete,age,gender
0,013f5fc1-a,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:04:36,dd,None,None,None,None
1,123,strin,strin,stri111n,NaT,None,None,None,None,None
2,24203a37-7,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:24,dd,None,None,None,None
3,6ae3a923-e,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:00:08,dd,None,None,None,None
4,a1d83841-5,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:43,dd,None,None,None,None
5,ss2,string,string,string,2021-12-19 08:03:22,good,None,None,None,None
6,strinfffg,string,string,string,2021-12-19 09:47:45,string,None,None,None,None
7,string,string,string,string,2021-12-19 08:36:34,string,None,None,None,None


In [94]:
data = data.append({'id':'asd', 'father_url':'asd', 'mother_url':'df', 'baby_url':'asff', 'created_time':datetime.now(), 'comment':'com1', 'closed_at': datetime.now(), 'complete':False,'age': 20, 'gender':'female'}, ignore_index=True)
data = data.append({'id':'asdf', 'father_url':'aswd', 'mother_url':'awdf', 'baby_url':'asff', 'created_time':datetime.now(), 'comment':'com2', 'closed_at': datetime.now(), 'complete':True,'age': 30, 'gender':'male'}, ignore_index=True)
data = data.append({'id':'assd', 'father_url':'absd', 'mother_url':'dfs', 'baby_url':'asff', 'created_time':datetime.now(), 'comment':'com3', 'closed_at': datetime.now(), 'complete':False,'age': 40, 'gender':'female'}, ignore_index=True)


data


,id,father_url,mother_url,baby_url,created_time,comment,closed_at,complete,age,gender
0,013f5fc1-a,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:04:36.000000,dd,None,None,None,None
1,123,strin,strin,stri111n,NaT,None,None,None,None,None
2,24203a37-7,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:24.000000,dd,None,None,None,None
3,6ae3a923-e,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:00:08.000000,dd,None,None,None,None
4,a1d83841-5,https://12war.s3.ap-northeast-2.amazonaws.com/...,https://12war.s3.ap-northeast-2.amazonaws.com/...,baby_url_test,2021-12-19 10:05:43.000000,dd,None,None,None,None
5,ss2,string,string,string,2021-12-19 08:03:22.000000,good,None,None,None,None
6,strinfffg,string,string,string,2021-12-19 09:47:45.000000,string,None,None,None,None
7,string,string,string,string,2021-12-19 08:36:34.000000,string,None,None,None,None
8,asd,asd,df,asff,2021-12-19 10:41:00.872791,com1,2021-12-19 10:41:00.872793,False,20,female
9,asdf,aswd,awdf,asff,2021-12-19 10:41:00.879591,com2,2021-12-19 10:41:00.879594,True,30,male


In [95]:
gen_age = calculate_bounce_rate(data)
gen_age

calculate bounce rate after click inference button
male 20
0
male 30
0
male 40
0
male 50
0
female 20
0
female 30
0
female 40
0
female 50
0
defaultdict(<class 'str'>, {'female': [0, 0, 0, 0], 'male': [0, 0, 0, 0]})


defaultdict(str, {'female': [0, 0, 0, 0], 'male': [0, 0, 0, 0]})

In [8]:
import pymysql

connect = pymysql.connect(
			user=os.getenv('MYSQL_USER'), 
			passwd=os.getenv('MYSQL_PASSWORD'), 
			host=os.getenv('MYSQL_SERVER'),
			db=os.getenv('MYSQL_DB'), 
			charset='utf8'
		)
cursor = connect.cursor()

In [10]:
connect.close()
# connect.dispose()

Error: Already closed